# Console Session Report (Automatically Generated)
**Created by:** Mark
**Date:** May 28, 2024
**Model Version:** models/gemini-pro

**Session outcome:** *Create the final dataset, 'data', starting with the two .csv files, 'BechdelData.csv' and 'tmdb_data_targets.csv'*

 Reading in the CSV Files and Setting the Index**


In [ ]:
data = pd.read_csv('BechdelData.csv', index_col=0).set_index('imdbid')
df = pd.read_csv('tmdb_data_targets.csv', index_col=0, converters={"genre_ids": ast.literal_eval})
df.set_index('imdbid', inplace=True)

 Adding the 'pass_fail' Column**


In [ ]:
data['pass_fail'] = pd.Series()
for i in data.index:
    try:
        if data['rating'][i] == 3:
            data['pass_fail'][i] = 'pass'
        else:
            data['pass_fail'][i] = 'fail'
    except ValueError: pass

 Joining the Two DataFrames**


In [ ]:
data = data.join(df, on='imdbid', lsuffix='bechdel', rsuffix='tmdb')

 Getting the Genre Data from the TMDb API**


In [ ]:
url = "https://api.themoviedb.org/3/genre/movie/list"
auth = 'Bearer ' + open('tmdbauth.txt').read()
headers = {
    'accept' : 'application/json',
    'Authorization' : auth
}
response = requests.get(url, headers=headers)
genre_dict = {}
for genre in json.loads(response.text)['genres']:
    genre_dict[genre['id']] = genre['name']

 Adding the Genre Columns to the DataFrame**


In [ ]:
for i in genre_dict.keys():
    data[genre_dict[i]] = pd.Series()
for i in data.index:
    try:
        for j in data['genre_ids'][i]:
            data[genre_dict[j]][i] = 1
    except (TypeError, KeyError): continue

 Filling in Missing Values and Summing the Genre Columns**


In [ ]:
data.loc[:,'Action':] = data.loc[:,'Action':].fillna(0)
data.loc[:,'Action':].sum()